In [1]:
import numpy as np
import pandas as pd
from scipy import *  
from scipy.linalg import norm, pinv  
from matplotlib import pyplot as plt  
import warnings
warnings.filterwarnings("ignore")

In [303]:
# data = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\data.xlsx')
# data

In [2]:
data = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\dl.xlsx')

In [3]:
# data = data.drop(columns = ['A电压','B电压','C电压','时间','视在功率','综合倍率','有功功率','无功功率','CT变比','PT变比'])
data = data.drop(columns = ['时间','视在功率','综合倍率','有功功率','无功功率','CT变比','PT变比'])

In [4]:
for i in range(data.shape[0]):
    data['秒分'][i] = str(data['秒分'].values[i].strftime('%H:%M'))

In [314]:
uniques = np.unique(data['变压器编号'],axis=0)
print(uniques)

[ 68  72  74  76 134 139 168 185 227 230]


In [315]:
# transformer_1 = data.loc[data['变压器编号'] == 1].copy()
# transformer_2 = data.loc[data['变压器编号'] == 2].copy()
# transformer_3 = data.loc[data['变压器编号'] == 3].copy()
# transformer_4 = data.loc[data['变压器编号'] == 4].copy()
# transformer_5 = data.loc[data['变压器编号'] == 5].copy()

# transformer_6 = data.loc[data['变压器编号'] == 6].copy()
# transformer_7 = data.loc[data['变压器编号'] == 7].copy()
# transformer_8 = data.loc[data['变压器编号'] == 8].copy()
# transformer_9 = data.loc[data['变压器编号'] == 9].copy()
# transformer_10 = data.loc[data['变压器编号'] == 10].copy()

#new
transformer_68 = data.loc[data['变压器编号'] == 68].copy()
transformer_72 = data.loc[data['变压器编号'] == 72].copy()
transformer_74 = data.loc[data['变压器编号'] == 74].copy()
transformer_76 = data.loc[data['变压器编号'] == 76].copy()
transformer_134 = data.loc[data['变压器编号'] == 134].copy()
transformer_139 = data.loc[data['变压器编号'] == 139].copy()
transformer_168 = data.loc[data['变压器编号'] == 168].copy()
transformer_185 = data.loc[data['变压器编号'] == 185].copy()
transformer_227 = data.loc[data['变压器编号'] == 227].copy()
transformer_230 = data.loc[data['变压器编号'] == 230].copy()

In [316]:
# 负荷值的夹角特征θ_i
'''
负荷值的夹角特征
'''
def load_theta(transformer):
    transformer['负荷A'] = transformer['A电流']*transformer['A电压']
    transformer['负荷B'] = transformer['B电流']*transformer['B电压']
    transformer['负荷C'] = transformer['C电流']*transformer['C电压']
    
    transformer['负荷A_dload'] = (transformer['负荷A'] - transformer['负荷A'].shift(-1,fill_value=0))
    transformer['负荷B_dload'] = (transformer['负荷B'] - transformer['负荷B'].shift(-1,fill_value=0))
    transformer['负荷C_dload'] = (transformer['负荷C'] - transformer['负荷C'].shift(-1,fill_value=0))
    
    transformer['θ_A'] =  transformer['负荷A_dload']
    transformer['θ_B'] =  transformer['负荷B_dload']
    transformer['θ_C'] =  transformer['负荷C_dload']
    
    for i in range(transformer.shape[0]):
        transformer['θ_A'][i] =  math.atan2(transformer['负荷A_dload'][i] , transformer['负荷A'].mean())
        transformer['θ_B'][i] =  math.atan2(transformer['负荷B_dload'][i] , transformer['负荷B'].mean())
        transformer['θ_C'][i] =  math.atan2(transformer['负荷C_dload'][i] , transformer['负荷C'].mean())
    transformer = transformer.drop(columns = ['负荷A','负荷B','负荷C','负荷A_dload','负荷B_dload','负荷C_dload'])
    return transformer
# 电流骤降幅值Isa_i 
'''
电流骤降幅值ISAi表示第i时刻电流变化值相比于日平均电流的骤降水平
'''
def Isa(transformer):
    #transformer = transformer.loc[data['年月日'] == '2021-01-01'].copy()
    #transformer = transformer.sort_values(by ='秒分').set_index('年月日')
    transformer['Isa_A'] = (transformer['A电流'] - transformer['A电流'].shift(-1,fill_value=0))*100/transformer['A电流'].mean()
    transformer['Isa_B'] = (transformer['B电流'] - transformer['B电流'].shift(-1,fill_value=0))*100/transformer['B电流'].mean()
    transformer['Isa_C'] = (transformer['C电流'] - transformer['C电流'].shift(-1,fill_value=0))*100/transformer['C电流'].mean()
    return transformer

# 三相电流不平衡度
'''
三相电流不平衡度εi反映了三相电流幅值的偏差程度
'''
def Unbalance(transformer):
    transformer['Iave'] = transformer[['A电流','B电流','C电流']].mean(axis = 1)
    transformer['Ia - Iave'] = transformer.apply(lambda x: x['A电流'] - x['Iave'], axis=1).abs()
    transformer['Ib - Iave'] = transformer.apply(lambda x: x['B电流'] - x['Iave'], axis=1).abs()
    transformer['Ic - Iave'] = transformer.apply(lambda x: x['C电流'] - x['Iave'], axis=1).abs()
    transformer['三相电流不平衡度'] = transformer.apply(lambda x: max(x['Ib - Iave'],x['Ia - Iave'],x['Ic - Iave'])/x['Iave'], axis=1)
    transformer = transformer.drop(columns = ['Iave','Ia - Iave','Ib - Iave','Ic - Iave'])
    return transformer

# 电流跌落深度
'''
电流跌落深度ISDi表示第i−1时刻与第i时刻电流有效值差值与第i−1时刻电流有效值之比，
反映了第i时刻电流较上一时刻电流的跌落程度，当ISDi为正时，电流跌落；当ISDi为负时，电流增大。
'''
def ISD(transformer):
    transformer['Isd_A'] = (transformer['A电流'] - transformer['A电流'].shift(-1,fill_value=0))*100/transformer['A电流']
    transformer['Isd_B'] = (transformer['B电流'] - transformer['B电流'].shift(-1,fill_value=0))*100/transformer['B电流']
    transformer['Isd_C'] = (transformer['C电流'] - transformer['C电流'].shift(-1,fill_value=0))*100/transformer['C电流']
    return transformer

#电流幅值占比
'''
电流幅值占比IARi表示第i时刻电
'''
def IAR(transformer):
    transformer['Iar_A'] = transformer['A电流']*100/transformer['额定容量']
    transformer['Iar_B'] = transformer['B电流']*100/transformer['额定容量']
    transformer['Iar_C'] = transformer['C电流']*100/transformer['额定容量']
    return transformer

# 与前3个时刻的峰谷差
'''
通过数据分析，电流发生连续骤降时通常在3个时隙内完成，故在此选取第i时刻与前3个时刻电流有效值差值的最大值与额定电流之比为峰谷差pvdi
'''
def PVD(transformer):
    transformer['Pvd_A_1'] = (transformer['A电流'] - transformer['A电流'].shift(-1,fill_value=0)).abs()
    transformer['Pvd_B_1'] = (transformer['B电流'] - transformer['B电流'].shift(-1,fill_value=0)).abs()
    transformer['Pvd_C_1'] = (transformer['C电流'] - transformer['C电流'].shift(-1,fill_value=0)).abs()

    transformer['Pvd_A_2'] = (transformer['A电流'] - transformer['A电流'].shift(-2,fill_value=0)).abs()
    transformer['Pvd_B_2'] = (transformer['B电流'] - transformer['B电流'].shift(-2,fill_value=0)).abs()
    transformer['Pvd_C_2'] = (transformer['C电流'] - transformer['C电流'].shift(-2,fill_value=0)).abs()

    transformer['Pvd_A_3'] = (transformer['A电流'] - transformer['A电流'].shift(-3,fill_value=0)).abs()
    transformer['Pvd_B_3'] = (transformer['B电流'] - transformer['B电流'].shift(-3,fill_value=0)).abs()
    transformer['Pvd_C_3'] = (transformer['C电流'] - transformer['C电流'].shift(-3,fill_value=0)).abs()

    transformer['Pvd_A'] = transformer.apply(lambda x: max(x['Pvd_A_1'],x['Pvd_A_2'],x['Pvd_A_3'])/x['额定容量'], axis=1)
    transformer['Pvd_B'] = transformer.apply(lambda x: max(x['Pvd_B_1'],x['Pvd_B_2'],x['Pvd_B_3'])/x['额定容量'], axis=1)
    transformer['Pvd_C'] = transformer.apply(lambda x: max(x['Pvd_C_1'],x['Pvd_C_2'],x['Pvd_C_3'])/x['额定容量'], axis=1)
    
    transformer = transformer.drop(columns = ['Pvd_A_1','Pvd_A_2','Pvd_A_3','Pvd_B_1','Pvd_B_2','Pvd_B_3','Pvd_C_1','Pvd_C_2','Pvd_C_3'])
    
    return transformer

# 负载率差值。
'''
负载率差值Δηi表示第i时刻与第i−1时刻的负载率差值，反映了负载率的状态变化
'''
def load_diff(transformer):
    transformer['负载率差值'] = transformer['负载率'].shift(-1,fill_value=0) - transformer['负载率']
    return transformer

In [358]:
trans = transformer_230

In [359]:
import datetime

# start = '2021-06-30'
# end = '2021-07-19'

start = '2022-04-24'
end = '2022-06-01'

datestart = datetime.datetime.strptime(start, '%Y-%m-%d')
dateend = datetime.datetime.strptime(end, '%Y-%m-%d')

transformer_to_excel = pd.DataFrame()

while datestart < dateend:
    datestart += datetime.timedelta(days=1)
    
    transformer_day = trans.loc[trans['年月日'] == datestart.strftime('%Y-%m-%d')]
    transformer_day = transformer_day.sort_values(by ='秒分')
    
    
    while(transformer_day.shape[0]==0):
        datestart += datetime.timedelta(days=1)
        transformer_day = trans.loc[trans['年月日'] == datestart.strftime('%Y-%m-%d')]
        transformer_day = transformer_day.sort_values(by ='秒分').set_index('年月日')
        #print(datestart)

        #transformer_day = transformer_day.drop(index = [datestart.strftime('%Y-%m-%d')])
    
    transformer_day = load_theta(transformer_day.reset_index())
    transformer_day = Isa(transformer_day)
    transformer_day = Unbalance(transformer_day)
    transformer_day = ISD(transformer_day)
    transformer_day = IAR(transformer_day)
    transformer_day = PVD(transformer_day)
    transformer_day = load_diff(transformer_day)

    
    df = transformer_day.drop(columns = ['序号','额定容量','A电流','B电流','C电流','变压器编号'])
    
    
    transformer_to_excel = transformer_to_excel.append(df)

    print(datestart.strftime('%Y-%m-%d'))
#     print(transformer_day)

    #transformer_day.to_excel(datestart.strftime('%Y-%m-%d')+'.xls')
    #print(datestart.strftime('%Y-%m-%d'))


2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-30
2022-05-31
2022-06-01


In [360]:
transformer_to_excel = transformer_to_excel.reset_index()
transformer_to_excel = transformer_to_excel.drop(columns = ['index'])

In [361]:
feature = pd.DataFrame(transformer_to_excel)
feature.to_excel('tran230_feature.xlsx')

# label

In [426]:
trans1_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran1_feature.xlsx')
trans2_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran2_feature.xlsx')
trans3_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran3_feature.xlsx')
trans4_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran4_feature.xlsx')
trans5_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran5_feature.xlsx')

trans6_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran6_feature.xlsx')
trans7_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran7_feature.xlsx')
trans8_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran8_feature.xlsx')
trans9_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran9_feature.xlsx')
trans10_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran10_feature.xlsx')

trans68_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran68_feature.xlsx')
trans72_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran72_feature.xlsx')
trans74_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran74_feature.xlsx')
trans76_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran76_feature.xlsx')
trans134_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran134_feature.xlsx')

trans139_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran139_feature.xlsx')
trans168_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran168_feature.xlsx')
trans185_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran185_feature.xlsx')
trans227_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran227_feature.xlsx')
trans230_feature = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\tran230_feature.xlsx')

In [427]:
trans1_feature.drop(trans1_feature.tail(1).index,inplace=True)
trans2_feature.drop(trans2_feature.tail(1).index,inplace=True)
trans3_feature.drop(trans3_feature.tail(1).index,inplace=True)
trans4_feature.drop(trans4_feature.tail(1).index,inplace=True)
trans5_feature.drop(trans5_feature.tail(1).index,inplace=True)
trans6_feature.drop(trans6_feature.tail(1).index,inplace=True)
trans7_feature.drop(trans7_feature.tail(1).index,inplace=True)
trans8_feature.drop(trans8_feature.tail(1).index,inplace=True)
trans9_feature.drop(trans9_feature.tail(1).index,inplace=True)
trans10_feature.drop(trans10_feature.tail(1).index,inplace=True)

trans68_feature.drop(trans68_feature.tail(1).index,inplace=True)
trans72_feature.drop(trans72_feature.tail(1).index,inplace=True)
trans74_feature.drop(trans74_feature.tail(1).index,inplace=True)
trans76_feature.drop(trans76_feature.tail(1).index,inplace=True)
trans134_feature.drop(trans134_feature.tail(1).index,inplace=True)
trans139_feature.drop(trans139_feature.tail(1).index,inplace=True)
trans168_feature.drop(trans168_feature.tail(1).index,inplace=True)
trans185_feature.drop(trans185_feature.tail(1).index,inplace=True)
trans227_feature.drop(trans227_feature.tail(1).index,inplace=True)
trans230_feature.drop(trans230_feature.tail(1).index,inplace=True) 

In [428]:
trans1_feature['label'] = pd.DataFrame([0]*trans1_feature.shape[0])
trans2_feature['label'] = pd.DataFrame([0]*trans2_feature.shape[0])
trans3_feature['label'] = pd.DataFrame([0]*trans3_feature.shape[0])
trans4_feature['label'] = pd.DataFrame([0]*trans4_feature.shape[0])
trans5_feature['label'] = pd.DataFrame([0]*trans5_feature.shape[0])
trans6_feature['label'] = pd.DataFrame([0]*trans6_feature.shape[0])
trans7_feature['label'] = pd.DataFrame([0]*trans7_feature.shape[0])
trans8_feature['label'] = pd.DataFrame([0]*trans8_feature.shape[0])
trans9_feature['label'] = pd.DataFrame([0]*trans9_feature.shape[0])
trans10_feature['label'] = pd.DataFrame([0]*trans10_feature.shape[0])

trans68_feature['label'] = pd.DataFrame([0]*trans68_feature.shape[0])
trans72_feature['label'] = pd.DataFrame([0]*trans72_feature.shape[0])
trans74_feature['label'] = pd.DataFrame([0]*trans74_feature.shape[0])
trans76_feature['label'] = pd.DataFrame([0]*trans76_feature.shape[0])
trans134_feature['label'] = pd.DataFrame([0]*trans134_feature.shape[0])
trans139_feature['label'] = pd.DataFrame([0]*trans139_feature.shape[0])
trans168_feature['label'] = pd.DataFrame([0]*trans168_feature.shape[0])
trans185_feature['label'] = pd.DataFrame([0]*trans185_feature.shape[0])
trans227_feature['label'] = pd.DataFrame([0]*trans227_feature.shape[0])
trans230_feature['label'] = pd.DataFrame([0]*trans230_feature.shape[0])

In [431]:
trans1_feature_A = trans1_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans1_feature_B = trans1_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans1_feature_C = trans1_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [432]:
trans2_feature_A = trans2_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans2_feature_B = trans2_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans2_feature_C = trans2_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [433]:
trans3_feature_A = trans3_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans3_feature_B = trans3_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans3_feature_C = trans3_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [434]:
trans4_feature_A = trans4_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans4_feature_B = trans4_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans4_feature_C = trans4_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [435]:
trans5_feature_A = trans5_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans5_feature_B = trans5_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans5_feature_C = trans5_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [436]:
trans6_feature_A = trans6_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans6_feature_B = trans6_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans6_feature_C = trans6_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [437]:
trans7_feature_A = trans7_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans7_feature_B = trans7_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans7_feature_C = trans7_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [438]:
trans8_feature_A = trans8_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans8_feature_B = trans8_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans8_feature_C = trans8_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [439]:
trans9_feature_A = trans9_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans9_feature_B = trans9_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans9_feature_C = trans9_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [440]:
trans10_feature_A = trans10_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans10_feature_B = trans10_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans10_feature_C = trans10_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [441]:
trans68_feature_A = trans68_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans68_feature_B = trans68_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans68_feature_C = trans68_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [442]:
trans72_feature_A = trans72_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans72_feature_B = trans72_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans72_feature_C = trans72_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [443]:
trans74_feature_A = trans74_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans74_feature_B = trans74_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans74_feature_C = trans74_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [444]:
trans76_feature_A = trans76_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans76_feature_B = trans76_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans76_feature_C = trans76_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [445]:
trans134_feature_A = trans134_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans134_feature_B = trans134_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans134_feature_C = trans134_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [446]:
trans139_feature_A = trans139_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans139_feature_B = trans139_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans139_feature_C = trans139_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [447]:
trans168_feature_A = trans168_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans168_feature_B = trans168_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans168_feature_C = trans168_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [448]:
trans185_feature_A = trans185_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans185_feature_B = trans185_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans185_feature_C = trans185_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [449]:
trans227_feature_A = trans227_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans227_feature_B = trans227_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans227_feature_C = trans227_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [450]:
trans230_feature_A = trans230_feature[['年月日','秒分','负载率','Isa_A','三相电流不平衡度','Isd_A','Iar_A','Pvd_A','负载率差值','label','θ_A']]
trans230_feature_B = trans230_feature[['年月日','秒分','负载率','Isa_B','三相电流不平衡度','Isd_B','Iar_B','Pvd_B','负载率差值','label','θ_B']]
trans230_feature_C = trans230_feature[['年月日','秒分','负载率','Isa_C','三相电流不平衡度','Isd_C','Iar_C','Pvd_C','负载率差值','label','θ_C']]

In [464]:
#label_data = pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\data.xlsx', sheet_name='sheet2', header=0)
label_data =  pd.read_excel(io=r'C:\Users\Serendipity\Attention_RBF\骤降补充.xlsx')

In [465]:
for i in range(label_data.shape[0]):
    label_data['骤降时分'][i] = str(label_data['骤降时分'].values[i].strftime('%H:%M'))

In [466]:
label_data

,骤降时间,骤降日期,骤降时分,骤降电流,变压器编号
0,2022-03-01 01:30:00.000000,2022-03-01,01:30,B,68
1,2022-03-01 16:30:00.000000,2022-03-01,16:30,A,68
2,2022-03-02 01:14:59.999997,2022-03-02,01:15,B,68
3,2022-03-10 16:45:00.000003,2022-03-10,16:45,A,68
4,2022-03-10 17:15:00.000000,2022-03-10,17:15,B,68
...,...,...,...,...,...
6283,2022-06-29 22:45:00.000003,2022-06-29,22:45,A,76
6284,2022-06-29 22:59:59.999997,2022-06-29,23:00,A,76
6285,2022-06-29 22:59:59.999997,2022-06-29,23:00,B,76
6286,2022-06-29 22:59:59.999997,2022-06-29,23:00,C,76


In [467]:
def label_(trans_feature,I_type,df_trans):
    for i in range(df_trans.shape[0]):
        if(df_trans['骤降电流'][i] == I_type):
            date = df_trans['骤降日期'][i].strftime("%Y-%m-%d")
            #time = df_trans['骤降时分'][i].strftime("%H:%M")
            time = df_trans['骤降时分'][i] #str
        
            idx = trans_feature[(trans_feature.年月日 == date) & (trans_feature.秒分 ==time)].index
            trans_feature['label'][idx] = 1
            #print(idx)
    #print('---')
    return trans_feature

# 每种类型特征加标签

In [454]:
# 变压器1
df_trans = label_data.loc[label_data['变压器编号'] == 1].reset_index()
trans1_feature_A = label_(trans1_feature_A,'A',df_trans)
trans1_feature_B = label_(trans1_feature_B,'B',df_trans)
trans1_feature_C = label_(trans1_feature_C,'C',df_trans)

In [455]:
# 变压器2
df_trans = label_data.loc[label_data['变压器编号'] == 2].reset_index()
trans2_feature_A = label_(trans2_feature_A,'A',df_trans)
trans2_feature_B = label_(trans2_feature_B,'B',df_trans)
trans2_feature_C = label_(trans2_feature_C,'C',df_trans)

In [456]:
# 变压器3
df_trans = label_data.loc[label_data['变压器编号'] == 3].reset_index()
trans3_feature_A = label_(trans3_feature_A,'A',df_trans)
trans3_feature_B = label_(trans3_feature_B,'B',df_trans)
trans3_feature_C = label_(trans3_feature_C,'C',df_trans)

In [457]:
# 变压器4
df_trans = label_data.loc[label_data['变压器编号'] == 4].reset_index()
trans4_feature_A = label_(trans4_feature_A,'A',df_trans)
trans4_feature_B = label_(trans4_feature_B,'B',df_trans)
trans4_feature_C = label_(trans4_feature_C,'C',df_trans)

In [458]:
# 变压器5
df_trans = label_data.loc[label_data['变压器编号'] == 5].reset_index()
trans5_feature_A = label_(trans5_feature_A,'A',df_trans)
trans5_feature_B = label_(trans5_feature_B,'B',df_trans)
trans5_feature_C = label_(trans5_feature_C,'C',df_trans)

In [459]:
# 变压器6
df_trans = label_data.loc[label_data['变压器编号'] == 6].reset_index()
trans6_feature_A = label_(trans6_feature_A,'A',df_trans)
trans6_feature_B = label_(trans6_feature_B,'B',df_trans)
trans6_feature_C = label_(trans6_feature_C,'C',df_trans)

In [460]:
# 变压器7
df_trans = label_data.loc[label_data['变压器编号'] == 7].reset_index()
trans7_feature_A = label_(trans7_feature_A,'A',df_trans)
trans7_feature_B = label_(trans7_feature_B,'B',df_trans)
trans7_feature_C = label_(trans7_feature_C,'C',df_trans)

In [461]:
# 变压器8
df_trans = label_data.loc[label_data['变压器编号'] == 8].reset_index()
trans8_feature_A = label_(trans8_feature_A,'A',df_trans)
trans8_feature_B = label_(trans8_feature_B,'B',df_trans)
trans8_feature_C = label_(trans8_feature_C,'C',df_trans)

In [462]:
# 变压器9
df_trans = label_data.loc[label_data['变压器编号'] == 9].reset_index()
trans9_feature_A = label_(trans9_feature_A,'A',df_trans)
trans9_feature_B = label_(trans9_feature_B,'B',df_trans)
trans9_feature_C = label_(trans9_feature_C,'C',df_trans)

In [463]:
# 变压器10
df_trans = label_data.loc[label_data['变压器编号'] == 10].reset_index()
trans10_feature_A = label_(trans10_feature_A,'A',df_trans)
trans10_feature_B = label_(trans10_feature_B,'B',df_trans)
trans10_feature_C = label_(trans10_feature_C,'C',df_trans)

In [468]:
# 变压器68
df_trans = label_data.loc[label_data['变压器编号'] == 68].reset_index()
trans68_feature_A = label_(trans68_feature_A,'A',df_trans)
trans68_feature_B = label_(trans68_feature_B,'B',df_trans)
trans68_feature_C = label_(trans68_feature_C,'C',df_trans)

In [469]:
# 变压器72
df_trans = label_data.loc[label_data['变压器编号'] == 72].reset_index()
trans72_feature_A = label_(trans72_feature_A,'A',df_trans)
trans72_feature_B = label_(trans72_feature_B,'B',df_trans)
trans72_feature_C = label_(trans72_feature_C,'C',df_trans)

In [470]:
# 变压器74
df_trans = label_data.loc[label_data['变压器编号'] == 74].reset_index()
trans74_feature_A = label_(trans74_feature_A,'A',df_trans)
trans74_feature_B = label_(trans74_feature_B,'B',df_trans)
trans74_feature_C = label_(trans74_feature_C,'C',df_trans)

In [471]:
# 变压器76
df_trans = label_data.loc[label_data['变压器编号'] == 76].reset_index()
trans76_feature_A = label_(trans76_feature_A,'A',df_trans)
trans76_feature_B = label_(trans76_feature_B,'B',df_trans)
trans76_feature_C = label_(trans76_feature_C,'C',df_trans)

In [472]:
# 变压器134
df_trans = label_data.loc[label_data['变压器编号'] == 134].reset_index()
trans134_feature_A = label_(trans134_feature_A,'A',df_trans)
trans134_feature_B = label_(trans134_feature_B,'B',df_trans)
trans134_feature_C = label_(trans134_feature_C,'C',df_trans)

In [473]:
# 变压器139
df_trans = label_data.loc[label_data['变压器编号'] == 139].reset_index()
trans139_feature_A = label_(trans139_feature_A,'A',df_trans)
trans139_feature_B = label_(trans139_feature_B,'B',df_trans)
trans139_feature_C = label_(trans139_feature_C,'C',df_trans)

In [474]:
# 变压器168
df_trans = label_data.loc[label_data['变压器编号'] == 168].reset_index()
trans168_feature_A = label_(trans168_feature_A,'A',df_trans)
trans168_feature_B = label_(trans168_feature_B,'B',df_trans)
trans168_feature_C = label_(trans168_feature_C,'C',df_trans)

In [475]:
# 变压器185
df_trans = label_data.loc[label_data['变压器编号'] == 185].reset_index()
trans185_feature_A = label_(trans185_feature_A,'A',df_trans)
trans185_feature_B = label_(trans185_feature_B,'B',df_trans)
trans185_feature_C = label_(trans185_feature_C,'C',df_trans)

In [476]:
# 变压器227
df_trans = label_data.loc[label_data['变压器编号'] == 227].reset_index()
trans227_feature_A = label_(trans227_feature_A,'A',df_trans)
trans227_feature_B = label_(trans227_feature_B,'B',df_trans)
trans227_feature_C = label_(trans227_feature_C,'C',df_trans)

In [477]:
# 变压器230
df_trans = label_data.loc[label_data['变压器编号'] == 230].reset_index()
trans230_feature_A = label_(trans230_feature_A,'A',df_trans)
trans230_feature_B = label_(trans230_feature_B,'B',df_trans)
trans230_feature_C = label_(trans230_feature_C,'C',df_trans)

# 特征集合成

In [478]:
Feature_A = pd.concat([
    trans1_feature_A,
    trans2_feature_A,
    trans3_feature_A,
    trans4_feature_A,
    trans5_feature_A,
    trans6_feature_A,
    trans7_feature_A,
    trans8_feature_A,
    trans9_feature_A,
    trans10_feature_A,
    
    trans68_feature_A,
    trans72_feature_A,
    trans74_feature_A,
    trans76_feature_A,
    trans134_feature_A,
    trans139_feature_A,
    trans168_feature_A,
    trans185_feature_A,
    trans227_feature_A,
    trans230_feature_A,
]) 

In [479]:
Feature_B = pd.concat([
    trans1_feature_B,
    trans2_feature_B,
    trans3_feature_B,
    trans4_feature_B,
    trans5_feature_B,
    trans6_feature_B,
    trans7_feature_B,
    trans8_feature_B,
    trans9_feature_B,
    trans10_feature_B,
    
    trans68_feature_B,
    trans72_feature_B,
    trans74_feature_B,
    trans76_feature_B,
    trans134_feature_B,
    trans139_feature_B,
    trans168_feature_B,
    trans185_feature_B,
    trans227_feature_B,
    trans230_feature_B,
]) 

In [480]:
Feature_C = pd.concat([
    trans1_feature_C,
    trans2_feature_C,
    trans3_feature_C,
    trans4_feature_C,
    trans5_feature_C,
    trans6_feature_C,
    trans7_feature_C,
    trans8_feature_C,
    trans9_feature_C,
    trans10_feature_C,
    
    trans68_feature_C,
    trans72_feature_C,
    trans74_feature_C,
    trans76_feature_C,
    trans134_feature_C,
    trans139_feature_C,
    trans168_feature_C,
    trans185_feature_C,
    trans227_feature_C,
    trans230_feature_C,
]) 

In [481]:
Feature_A.to_excel('Feature_A.xlsx')
Feature_B.to_excel('Feature_B.xlsx')
Feature_C.to_excel('Feature_C.xlsx')

In [485]:
Feature_A = Feature_A.rename(columns={"Isa_A": "Isa", "Isd_A": "Isd","Iar_A": "Iar","Pvd_A": "Pvd","θ_A":"θ"})
Feature_B = Feature_B.rename(columns={"Isa_B": "Isa", "Isd_B": "Isd","Iar_B": "Iar","Pvd_B": "Pvd","θ_B":"θ"})
Feature_C = Feature_C.rename(columns={"Isa_C": "Isa", "Isd_C": "Isd","Iar_C": "Iar","Pvd_C": "Pvd","θ_C":"θ"})

In [486]:
Feature = pd.concat([
    Feature_A,
    Feature_B,
    Feature_C,
]) 

In [487]:
Feature

,年月日,秒分,负载率,Isa,三相电流不平衡度,Isd,Iar,Pvd,负载率差值,label,θ
0,2021-07-01,00:00,14.3738,0.000000,0.221106,0.000000,0.128571,0.000222,-1.6595,0,0.006229
1,2021-07-01,00:15,12.7143,-7.931262,0.372881,-6.172840,0.128571,0.000222,1.2278,0,-0.078344
2,2021-07-01,00:30,13.9421,-14.276272,0.316327,-10.465116,0.136508,0.000270,1.8539,0,-0.140728
3,2021-07-01,00:45,15.7960,38.070059,0.295455,25.263158,0.150794,0.000413,-3.3165,0,0.353894
4,2021-07-01,01:00,12.4795,3.172505,0.231214,2.816901,0.112698,0.000032,-0.2619,0,0.034775
...,...,...,...,...,...,...,...,...,...,...,...
3234,2022-06-01 00:00:00,22:30,31.1940,20.273783,0.354571,12.264151,0.132500,0.000188,-1.4550,0,0.194918
3235,2022-06-01 00:00:00,22:45,29.7390,3.119043,0.320700,2.150538,0.116250,0.000162,-1.3405,0,0.031242
3236,2022-06-01 00:00:00,23:00,28.3985,-17.154739,0.298780,-12.087912,0.113750,0.000138,0.8215,0,-0.163540
3237,2022-06-01 00:00:00,23:15,29.2200,34.309478,0.274336,21.568627,0.127500,0.001275,-3.1544,0,0.324957


In [488]:
label_y = Feature['label']
X_feat =  Feature[['负载率','Isa','三相电流不平衡度','Isd','Iar','Pvd','负载率差值','θ']]
# float64->float32
#X_feat[X_feat.select_dtypes(np.float64).columns] = X_feat.select_dtypes(np.float64).astype(np.float32)

In [489]:
from collections import Counter
# 查看所生成的样本类别分布，0和1样本比例9比1，属于类别不平衡数据
print(Counter(label_y))
# Counter({0: 900, 1: 100})

Counter({0: 149618, 1: 1834})


In [ ]:
# inf,nan数据填充
X_feat = (X_feat.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 

In [491]:
# 使用imlbearn库中上采样方法中的SMOTE接口
from imblearn.over_sampling import SMOTE
# 定义SMOTE模型，random_state相当于随机数种子的作用
smo = SMOTE(random_state=42)
X_smo, y_smo = smo.fit_resample(X_feat, label_y)

In [492]:
X_smo

,负载率,Isa,三相电流不平衡度,Isd,Iar,Pvd,负载率差值,θ
0,14.373800,0.000000,0.221106,0.000000,0.128571,0.000222,-1.659500,0.006229
1,12.714300,-7.931262,0.372881,-6.172840,0.128571,0.000222,1.227800,-0.078344
2,13.942100,-14.276272,0.316327,-10.465116,0.136508,0.000270,1.853900,-0.140728
3,15.796000,38.070059,0.295455,25.263158,0.150794,0.000413,-3.316500,0.353894
4,12.479500,3.172505,0.231214,2.816901,0.112698,0.000032,-0.261900,0.034775
...,...,...,...,...,...,...,...,...
299231,16.318183,-22.278998,0.251495,-20.864044,0.146751,0.000391,1.918612,-0.218812
299232,5.064354,-84.854526,0.279909,-151.887609,0.036940,0.000578,2.766760,-0.700622
299233,19.471195,-23.951392,0.386530,-17.749873,0.091627,0.000229,1.796548,-0.233068
299234,18.119446,-50.202186,0.568227,-132.136228,0.076000,0.001348,14.167156,-0.457158
